#### Markowitz Modern Portfolio Theory is a practical method that allows us to build a portfolio that maximizes returns while maintaining a reasonable level of risk. <br>

#### The key component of this theory is diversification. It explains that by investing in stocks that are uncorrelated, we can lower the overall risk of a portfolio while maximizing returns for that level of risk. <br>

#### in MPT, the risk and return of an investment is not viewed individually, but by how it affects the risk and return of the entire portfolio.


#### This project uses Markowitz's Theory to create an efficient portfolio using only technology stocks from the S&P 500 index.

#### Libraries used in project



In [ ]:
#import necessary libraries
import os
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import yfinance as yfin
import matplotlib.pyplot as plt

### Functions

Creating all the functions that will be used over the course of this project

In [ ]:
def get_dataframe_from_webpage(url, index):
    """
    desc: parses a webpage table into a dataframe
    params: webpage url, table index
    returns: dataframe of downloaded table
    """
    data = pd.read_html(url)[index]

    return data


In [ ]:
def dataframe_by_column_category(df, column_name, column_category):
    """
    desc: gets a slice of a dataframe by specifying a column category
    params: dataframe to be sliced, column name, column category
    returns: sliced dataframe
    """
    sliced_df = df.loc[df[ column_name ] == column_category]
    return sliced_df

In [ ]:
def get_items_from_dataframe_column(df, column_name):
    """
    desc: gets the row items of a dataframe column and saves them in a list
    params: dataframe, column name
    returns: row items of a dataframe column
    rtype: list
    """
    return df[column_name].to_list()

In [ ]:
def csv_from_yahoo_data(file_path, ticker, s_year, s_month, s_day, e_year, e_month, e_day):
    """
    desc: downloads company stock data for a specific period from yahoo finance and saves it as csv
    params: folder to dave data, company ticker, period start date, period end date
    returns: saved data as csv file
    rtype: .csv
    """
    start_date = dt.datetime(s_year, s_month, s_day)
    end_date = dt.datetime(e_year, s_month, s_day)

    try:
        print(f"downloading data for company:{ticker}")
        stock_data = yfin.download(ticker, start_date, end_date)

        stock_data.to_csv(file_path + ticker + ".csv")

    except Exception as e:
        print(f"Failed to download stock data for {ticker}")

    else:
        return stock_data

In [ ]:
def multiple_csv_from_yahoo_data(file_path, s_year, s_month, s_day, e_year, e_month, e_day, tickers):
    """
    desc: download and save multiple stock data from yahoo finance to csv
    params: list of tickers, period start date, period end date
    """
    for ticker in tickers:
        csv_from_yahoo_data(file_path, ticker, s_year, s_month, s_day, e_year, e_month, e_day)

In [ ]:
def count_csv_in_folder(file_path):
    """
    desc: returns the number of csv files present in a path
    params: folder path
    returns: count of csv files
    rtype: int
    """
    files = [file for file in os.listdir(file_path) if file.endswith(".csv")]
    return len(files)

In [ ]:
def df_from_csv(ticker):
    """
    desc:  creates dataframe from a csv file
    params: csv file path
    returns: pandas dataframe
    """
    df = pd.read_csv(file_path + ticker + ".csv", index_col='Date', parse_dates=True)
    return df


In [ ]:
def merge_df_by_column(column_name, tickers):
    """
    desc: merges multiple dataframes by column
    params: column to merge by, tickers for each df
    returns: merged dataframe
    rtype: pandas dataframe
    """
    merged_df = pd.DataFrame()

    for ticker in tickers:
        df = df_from_csv(ticker)
        merged_df[ticker] = df[column_name]

    return merged_df

In [ ]:
def get_returns_from_df(df):
    """
    desc: calculates returns (relative change in price) for all columns in a dataframe
    params: dataframe
    returns returns dataframe
    rtype: pandas dataframe
    """
    returns = df.pct_change()
    return returns

In [ ]:
def get_roi(df):
    """
    desc: get return on investment (roi) for dataframe
    params: dataframe
    returns: roi value
    rtype: float
    """
    start = df.loc[df.index[0]]['Adj Close']
    end = df.loc[df.index[-1]]['Adj Close']

    roi = (end - start) / start

    return roi


In [ ]:
def get_roi_for_multiple_stocks(tickers):
    """
    desc: create dataframe of rois for all stocks by their tickers
    params: tickers list
    returns: dataframe of rois
    rtype: pandas dataframe
    """
    tickers_list = []
    rois = []

    for ticker in tickers:
        df = df_from_csv(ticker)
        roi = get_roi(df)
        rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

### Data Preparation


Download and save the data containing the list of all S&P 500 companies

In [ ]:
#specify url to get the list of all companies  in s&p 500
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

#download the list of comapnies and save it in a variable called s_p_df
s_p_df = get_dataframe_from_webpage(url, 0)

s_p_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


#### Filter Technology Companies

The technology companies are labeled as "Information Technology" in the "GICS Sector" column. <br>
We are only interested in these companies so we filter them out by this label

In [ ]:
#Filter and save the list technology companies
tech = dataframe_by_column_category(s_p_df, "GICS Sector", "Information Technology" )


#### Download stock data for tech companies
Download stock data for each of the companies and save them as csv files. We will be using data between January 2021 and January 2023 for our analysis (2 years worh of data)

In [ ]:
#get the symbol for each company and save it in a list
tech_tickers = get_items_from_dataframe_column(tech, 'Symbol')
print(f"Company Symbols: {tech_tickers})")

#save the count of symbols
num_of_tickers = len(tech_tickers)

#specify file_path to store csv files
file_path = "/content/drive/MyDrive/Data_Science/Projects/MPT/Stock_Files/"

#download two years worth of data for each company
multiple_csv_from_yahoo_data(file_path, 2021, 1, 1, 2023, 1, 1, tech_tickers)

#print the number of csv files downloaded
num_of_files = count_csv_in_folder(file_path)

# compare the number of symbols with length of files downloaded
print(f"number of symbols = number of files: {num_of_tickers == num_of_files}")

Company Symbols: ['ACN', 'ADBE', 'AKAM', 'AMD', 'APH', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ANET', 'ADSK', 'AVGO', 'CDNS', 'CDW', 'CDAY', 'CSCO', 'CTSH', 'GLW', 'DXC', 'ENPH', 'EPAM', 'FFIV', 'FICO', 'FSLR', 'FTNT', 'IT', 'GEN', 'HPE', 'HPQ', 'IBM', 'INTC', 'INTU', 'JNPR', 'KEYS', 'KLAC', 'LRCX', 'MCHP', 'MU', 'MSFT', 'MPWR', 'MSI', 'NTAP', 'NVDA', 'NXPI', 'ON', 'ORCL', 'PANW', 'PAYC', 'PTC', 'QRVO', 'QCOM', 'ROP', 'CRM', 'STX', 'NOW', 'SWKS', 'SEDG', 'SNPS', 'TEL', 'TDY', 'TER', 'TXN', 'TRMB', 'TYL', 'VRSN', 'WDC', 'ZBRA'])
downloading data for company:ACN
[*********************100%%**********************]  1 of 1 completed
downloading data for company:ADBE
[*********************100%%**********************]  1 of 1 completed
downloading data for company:AKAM
[*********************100%%**********************]  1 of 1 completed
downloading data for company:AMD
[*********************100%%**********************]  1 of 1 completed
downloading data for company:APH
[*********************100%%****

#### Create DataFrames

Example dataframe using data from the first company in our list


In [ ]:
#Company name
print("Company Symbol: ", tech_tickers[0])
print()

#dataframe using company data
df_from_csv(tech_tickers[0]).head()

Company Symbol:  ACN



,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,261.000000,261.660004,254.050003,256.459991,247.030975,2505100
2021-01-05,256.399994,258.640015,255.139999,257.920013,248.437332,1809200
2021-01-06,259.200012,263.589996,256.950012,260.739990,251.153641,2535600
2021-01-07,260.359985,264.920013,260.339996,263.200012,253.523132,2068200
2021-01-08,263.809998,266.529999,261.290009,264.160004,254.447861,2010700


Now we have data for all our companies. We have to merge the data into one dataframe to be used for our analysis. <p>
 We merge the dataframes on the "Adj close" (Adjusted close) column which is an accurate indicator of the stock value at the end of each day

In [ ]:
#merge dataframes by Adjusted close price
merged_df = merge_df_by_column('Adj Close', tech_tickers)

#print first 5 rows of merged dataframe
merged_df.head()

,ACN,ADBE,AKAM,AMD,APH,ADI,ANSS,AAPL,AMAT,ANET,...,SNPS,TEL,TDY,TER,TXN,TRMB,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,247.030975,485.339996,105.239998,92.300003,62.649925,138.974472,357.670013,127.331673,84.861465,70.790001,...,254.699997,115.120789,362.390015,117.936989,150.813828,66.290001,431.609985,213.580002,52.230000,378.130005
2021-01-05,248.437332,485.690002,105.419998,92.769997,63.732117,141.458557,359.829987,128.905991,87.528320,71.502502,...,258.890015,117.331337,375.489990,120.212791,151.920151,66.849998,432.239990,206.500000,53.060001,380.570007
2021-01-06,251.153641,466.309998,101.529999,90.330002,65.316414,142.096222,349.899994,124.566811,88.729889,71.467499,...,249.699997,120.699783,368.890015,121.123131,152.673203,68.410004,433.730011,199.509995,53.410000,394.820007
2021-01-07,253.523132,477.739990,102.809998,95.160004,66.252342,148.101761,365.019989,128.817429,92.373642,73.992500,...,258.839996,123.120865,356.170013,128.544296,156.131638,71.839996,437.670013,200.000000,54.740002,409.100006
2021-01-08,254.447861,485.100006,103.459999,94.580002,65.984238,149.177246,374.660004,129.929291,93.350510,76.529999,...,262.799988,124.852913,368.329987,130.760727,159.125229,72.320000,436.390015,202.440002,51.820000,405.470001


#### Create Portfolio

From our analysis so far, we have 67 stocks in our dataframe, to create an efficient portfolio, we decide to use the top 20 performing stocks for our portfolio. We do this by calculating the return on investments for every stock and use the top 20 for our portfolio

In [ ]:
#get return on investment for all stocks in dataframe
roi_df = get_roi_for_multiple_stocks(tech_tickers)
print("first 5 rows of roi dataframe\n", roi_df.head(), "\n")

#get top 20 companies with highest return on investment
top_roi = roi_df.sort_values(by='ROI', ascending=False)[:20]
print("top 20 roi\n", top_roi, "\n")

#get tickers for top 20 companies
top_tickers = get_items_from_dataframe_column(top_roi, 'Ticker')

#create our portfolio dataframe by taking top 20 stocks from merged dataframe
port_df= pd.DataFrame()
for ticker in top_tickers:
    port_df[ticker] = merged_df[ticker]

print("dataframe of top 20 performing stocks\n")
port_df.head()

first 5 rows of roi dataframe
   Ticker       ROI
0    ACN  0.067705
1   ADBE -0.306610
2   AKAM -0.198974
3    AMD -0.298267
4    APH  0.208892 

top 20 roi
    Ticker       ROI
25     IT  1.126526
44     ON  0.909091
9    ANET  0.714225
24   FTNT  0.679376
40    MSI  0.581103
19   ENPH  0.538319
32   JNPR  0.494879
34   KLAC  0.484144
23   FSLR  0.479554
27    HPE  0.470158
13    CDW  0.434787
11   AVGO  0.397511
45   ORCL  0.324543
29    IBM  0.310813
33   KEYS  0.304086
57   SNPS  0.253593
4     APH  0.208892
46   PANW  0.190411
22   FICO  0.184697
12   CDNS  0.183265 

dataframe of top 20 performing stocks



,IT,ON,ANET,FTNT,MSI,ENPH,JNPR,KLAC,FSLR,HPE,CDW,AVGO,ORCL,IBM,KEYS,SNPS,APH,PANW,FICO,CDNS
Date,,,,,,,,,,,,,,,,,,,,
2021-01-04,158.070007,32.669998,70.790001,29.112,161.934006,172.240005,20.912638,251.684143,101.239998,10.684711,123.318031,395.125854,61.010807,103.558601,131.179993,254.699997,62.649925,117.220001,505.260010,135.759995
2021-01-05,159.710007,33.220001,71.502502,28.348,161.073563,181.479996,21.606327,256.962830,92.160004,10.878641,123.977455,397.802032,60.254753,105.396843,133.839996,258.890015,63.732117,116.716667,505.010010,136.139999
2021-01-06,163.789993,34.060001,71.467499,27.622,165.008301,203.380005,21.809814,256.402100,99.690002,11.118748,125.005432,395.348846,60.110653,108.028809,136.110001,249.699997,65.316414,115.093330,497.010010,130.639999
2021-01-07,166.199997,35.360001,73.992500,28.650,165.395020,213.759995,22.041048,268.951050,105.500000,11.054104,127.982681,412.009949,60.590996,107.778145,141.000000,258.839996,66.252342,121.036667,490.109985,137.050003
2021-01-08,169.100006,35.520000,76.529999,29.628,168.749695,207.410004,22.013296,273.775238,104.099998,10.970989,130.523514,414.100708,60.888813,107.393806,146.800003,262.799988,65.984238,122.183334,500.359985,136.190002


### Returns

Now we calculate returns for all stocks in our portfolio. Returns tells us how the price changes over the time frame of the dataset, and we can use the information presented for further analysis to build our portfolio

In [ ]:
#get returns for our merged dataframe
returns = get_returns_from_df(port_df)

#print top 5 rows of returns dataframe
returns.head()

,IT,ON,ANET,FTNT,MSI,ENPH,JNPR,KLAC,FSLR,HPE,CDW,AVGO,ORCL,IBM,KEYS,SNPS,APH,PANW,FICO,CDNS
Date,,,,,,,,,,,,,,,,,,,,
2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,0.010375,0.016835,0.010065,-0.026243,-0.005314,0.053646,0.033171,0.020973,-0.089688,0.018150,0.005347,0.006773,-0.012392,0.017751,0.020278,0.016451,0.017274,-0.004294,-0.000495,0.002799
2021-01-06,0.025546,0.025286,-0.000490,-0.025610,0.024428,0.120675,0.009418,-0.002182,0.081706,0.022071,0.008292,-0.006167,-0.002392,0.024972,0.016961,-0.035498,0.024859,-0.013908,-0.015841,-0.040400
2021-01-07,0.014714,0.038168,0.035331,0.037217,0.002344,0.051037,0.010602,0.048942,0.058281,-0.005814,0.023817,0.042143,0.007991,-0.002320,0.035927,0.036604,0.014329,0.051639,-0.013883,0.049066
2021-01-08,0.017449,0.004525,0.034294,0.034136,0.020283,-0.029706,-0.001259,0.017937,-0.013270,-0.007519,0.019853,0.005075,0.004915,-0.003566,0.041135,0.015299,-0.004047,0.009474,0.020914,-0.006275
